In [119]:
import torch
import torch.nn as nn
import numpy as np

### <u>Compare functions one by one</u>

#### 1. MaxPool1d

In [95]:
#Create some random input
random_tensor = torch.rand(3, 4)
random_tensor

tensor([[0.6735, 0.5949, 0.8780, 0.8296],
        [0.3949, 0.9947, 0.2762, 0.1802],
        [0.7275, 0.5064, 0.8903, 0.1068]])

In [96]:
#define the pytorch equivalent
pytorch_max_pool_fn = nn.MaxPool1d(kernel_size=2, stride=1, padding=0,dilation=1, return_indices=False, ceil_mode=False)
output_from_pytorch = pytorch_max_pool_fn(random_tensor)
output_from_pytorch

tensor([[0.6735, 0.8780, 0.8780],
        [0.9947, 0.9947, 0.2762],
        [0.7275, 0.8903, 0.8903]])

In [97]:
#define my custom implementation of maxPool1d ( assumes that x is 2D (C,W) shape)
def my_MaxPool1d(x, kernel_size=2, stride=1, padding=0, dilation=1):
    result = []
    for row in x:
        maxPoolForRow = []
        #k represents start point of the kernel
        for k in range(0, len(row) - kernel_size + 1, stride):
            maxPoolForRow.append(max(row[k:k + kernel_size]))
        result.append(maxPoolForRow)
    result = torch.tensor(result)
    return result


output_from_my_implementation = my_MaxPool1d(random_tensor)
output_from_my_implementation

tensor([[0.6735, 0.8780, 0.8780],
        [0.9947, 0.9947, 0.2762],
        [0.7275, 0.8903, 0.8903]])

In [98]:
#Assert that both are the same
torch.equal(output_from_pytorch, output_from_my_implementation)

True

#### 2. AvgPool1d

In [99]:
#Create some random input
random_tensor = torch.rand(3, 4)
random_tensor

tensor([[0.5099, 0.7110, 0.6018, 0.9501],
        [0.6111, 0.8375, 0.8109, 0.6189],
        [0.7721, 0.2097, 0.9601, 0.4213]])

In [100]:
#define the pytorch equivalent
pytorch_avg_pool_fn = nn.AvgPool1d(kernel_size=2, stride=1, padding=0,ceil_mode=False, count_include_pad=True)
output_from_pytorch = pytorch_avg_pool_fn(random_tensor)
output_from_pytorch


tensor([[0.6104, 0.6564, 0.7760],
        [0.7243, 0.8242, 0.7149],
        [0.4909, 0.5849, 0.6907]])

In [101]:
#define my custom implementation of avgPool1d ( assumes that x is 2D (C,W) shape)
def my_AvgPool1d(x, kernel_size=2, stride=1, padding=0, dilation=1):
    result = []
    for row in x:
        maxPoolForRow = []
        #k represents start point of the kernel
        for k in range(0, len(row) - kernel_size + 1, stride):
            maxPoolForRow.append(sum(row[k:k + kernel_size])/kernel_size)
        result.append(maxPoolForRow)
    result = torch.tensor(result)
    return result


output_from_my_implementation = my_AvgPool1d(random_tensor)
output_from_my_implementation

tensor([[0.6104, 0.6564, 0.7760],
        [0.7243, 0.8242, 0.7149],
        [0.4909, 0.5849, 0.6907]])

In [102]:
#Assert that both are the same
torch.equal(output_from_pytorch, output_from_my_implementation)

True

#### 3. Conv1d

In [103]:
#Loading the given filter and input tensors
pixel_input = torch.load("./pixel_input.pt") #Shape (1,1,32)
filter = torch.load("./filter.pt") #(Shape 3,1,1)

In [104]:
#define pytorch equivalent
pytorch_conv1d_output = nn.functional.conv1d(pixel_input, filter, bias = None,stride = 1, padding = 0, dilation = 1, groups = 1)

#Shape is (1,3,32)

In [120]:
#define custom implementation
def my_Conv1d(inp, filter, stride=1, padding=0, dilation=1, groups = 1):
    result = []

    for f in filter:
        #as f in this case is 1x1 ( but it could in theory be something like 1 x k)
        f = f[0]
        batch_row = []
        for batch in inp: #(1,32)
            conv_row = []
            for channel in batch: #(32,)
                #convolve f over this channel array using this filter by using dot product to convolve!
                for f_ind in range(0, len(channel) - len(f) + 1, stride):
                    f = np.array(f)
                    conv_row.append(np.dot(f, np.array(channel[f_ind: f_ind + len(f)])))
            batch_row.append(conv_row)
        result.append(batch_row)

    result = torch.tensor(result)

    #permute it so that the shapes are correct.
    result = result.permute(1,0,2)
    return result

my_conv1d_output = my_Conv1d(pixel_input, filter)

In [121]:
#Assert that both are the same
torch.equal(pytorch_conv1d_output, my_conv1d_output)

True

#### 4. Sigmoid



In [122]:
#Create some random input
random_tensor = torch.rand(2, 2)
random_tensor

tensor([[0.8103, 0.1572],
        [0.4445, 0.2277]])

In [123]:
#define the pytorch equivalent
pytorch_sigmoid_fn = nn.Sigmoid()
output_from_pytorch = pytorch_sigmoid_fn(random_tensor)
output_from_pytorch


tensor([[0.6922, 0.5392],
        [0.6093, 0.5567]])

In [124]:
#define my custom implementation of maxPool1d ( assumes that x is 2D (C,W) shape)
def my_Sigmoid(x):
    return 1/(1+np.exp(-x))

output_from_my_implementation = torch.tensor(my_Sigmoid(random_tensor))
output_from_my_implementation

C:\Users\paras\AppData\Local\Temp\ipykernel_5716\1711964653.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output_from_my_implementation = torch.tensor(my_Sigmoid(random_tensor))


tensor([[0.6922, 0.5392],
        [0.6093, 0.5567]])

In [125]:
#Assert that both are the same
torch.equal(pytorch_conv1d_output, my_conv1d_output)

True

#### 5. BatchNorm1d

In [126]:
#Create some random input
random_tensor = torch.rand(2,1,4) #(batchsize,numfeatures, actualfeatures)
random_tensor

tensor([[[0.8012, 0.8653, 0.0883, 0.3625]],

        [[0.9214, 0.6491, 0.8451, 0.5259]]])

In [127]:
#define the pytorch equivalent
pytorch_batchnorm_fn = nn.BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True, device=None, dtype=None)
output_from_pytorch = pytorch_batchnorm_fn(random_tensor)
output_from_pytorch = output_from_pytorch.data.detach()
output_from_pytorch

tensor([[[ 0.6205,  0.8562, -1.9999, -0.9918]],

        [[ 1.0624,  0.0616,  0.7822, -0.3914]]])

In [128]:
#define my custom implementation of batchnorm ( assumes that x is 3D (B,1,N) shape)
def my_BatchNorm1d(x, eps=1e-05, momentum=0.1, affine=True):
    #momentum is used only during training and not used during inference!
    # Compute batch mean and batch variance
    batch_mean = x.mean(dim=(0, 2), keepdim=True)
    batch_var = x.var(dim=(0, 2), unbiased=False, keepdim=True)

    # Normalize the input using batch mean and variance
    x_normalized = (x - batch_mean) / torch.sqrt(batch_var + eps)

    return x_normalized

output_from_my_implementation = my_BatchNorm1d(random_tensor)
output_from_my_implementation

tensor([[[ 0.6205,  0.8562, -1.9999, -0.9918]],

        [[ 1.0624,  0.0616,  0.7822, -0.3914]]])

In [129]:
#Assert that both are the same
torch.allclose(output_from_pytorch, output_from_my_implementation, rtol=1e-05, atol=1e-05)

True

#### 6. Linear

In [130]:
#Create some random input
random_tensor = torch.rand(4,3)
random_tensor

tensor([[0.5134, 0.2622, 0.7205],
        [0.0827, 0.9248, 0.4095],
        [0.5334, 0.2096, 0.2256],
        [0.7453, 0.0175, 0.8764]])

In [134]:
#define the pytorch equivalent
in_features = 3
out_features = 5
pytorch_linear_fn = nn.Linear(in_features, out_features, bias=True)
output_from_pytorch = pytorch_linear_fn(random_tensor)
output_from_pytorch = output_from_pytorch.data.detach()
output_from_pytorch

tensor([[ 0.5355, -0.1164, -0.4184,  0.6875, -0.5266],
        [ 0.0998, -0.3222, -0.7425,  0.4450, -0.7156],
        [ 0.6154, -0.0929, -0.4633,  0.4859, -0.3784],
        [ 0.7250, -0.0533, -0.2870,  0.8293, -0.4755]])

In [135]:
#define my custom implementation of batchnorm ( assumes that x is 3D (B,1,N) shape)
def my_Linear(x, in_features=3, out_features=5):
    weights = torch.randn(in_features, out_features)  # Corrected order
    biases = torch.randn(out_features)

    # Perform linear transformation
    return torch.tensor(np.matmul(x, weights)) + biases

output_from_my_implementation = my_Linear(random_tensor)
output_from_my_implementation

C:\Users\paras\AppData\Local\Temp\ipykernel_5716\2481492983.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(np.matmul(x, weights)) + biases


tensor([[-1.4493,  0.4005,  3.1598, -0.5529, -1.6624],
        [-1.5099,  1.1979,  1.3416, -0.4581, -1.6808],
        [-2.2352,  0.6601,  2.1254, -0.5660, -0.8809],
        [-1.4059, -0.0915,  3.9784, -0.5868, -1.7810]])

In [136]:
#Assert that both are the same size as the data inside may be random
output_from_pytorch.shape == output_from_my_implementation.shape

True